In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 33643, done.
remote: Counting objects: 100% (1440/1440), done.
remote: Compressing objects: 100% (816/816), done.
remote: Total 33643 (delta 718), reused 1254 (delta 609), pack-reused 32203
Receiving objects: 100% (33643/33643), 29.57 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (15992/15992), done.
Updating files: 100% (4450/4450), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.2 MB/s eta 0:00:00
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

url= "/content/drive/MyDrive/output/rest16/test.tsv"
import pandas as pd

# Step 3: 读取TSV文件
file_path = url
df = pd.read_csv(file_path, sep='\t')
df_first_200 = df.head(200)
df_last = df.iloc[201:]
print(df_first_200.head())
print(df_last.head())
# Assuming the original DataFrame is named df
# Extracting the 'review' and 'aspect' columns
reviews = df_first_200['review'].tolist()
aspects = df_first_200['aspect'].tolist()
sentiments = df_first_200['sentiment'].tolist()

# Combining the extracted columns into the desired format
dev = [(review, aspect, "neutral" if sentiment == 0 else ("positive" if sentiment == 1 else "negative")) for review, aspect, sentiment in zip(reviews, aspects, sentiments)]

# Converting to the desired format using dspy.Example
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

# Print the result to verify
# for example in dev:
#     print(example)

############train dataset###########
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

#
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

Mounted at /content/drive
                                        review                  aspect  \
0                    Serves really good sushi.                   sushi   
1       Not the biggest portions but adequate.                portions   
2            Green Tea creme brulee is a must!  Green Tea creme brulee   
3       – I can't say enough about this place.                   place   
4  It has great sushi and even better service.                   sushi   

   sentiment  
0          1  
1          0  
2          1  
3          1  
4          1  
                                                review       aspect  sentiment
201  i highly recommend this place to all that want...  indain food          1
202                 The lunch menu is an awesome deal!   lunch menu          1
203                          plenty of food,trust me .         food          1
204                 Fresh ingrediants and super tasty.  ingrediants          1
205                     Best food, phenomina

Define the Metric

In [ ]:
def detect_predicted_label(pred):
    # 将预测结果转换为小写
    pred_sentiment = pred.sentiment.lower()

    # 初始化 predicted_label 为 "unknown"
    predicted_label = "unknown"

    # 检查预测结果中的特定字符串并分类，以第一个出现的为准
    if "ositiv" in pred_sentiment:
        predicted_label = "positive"
    elif "egativ" in pred_sentiment:
        predicted_label = "negative"
    elif "eutra" in pred_sentiment:
        predicted_label = "neutral"
    elif "ositiv" and "egativ" and "eutra" in pred_sentiment:
        predicted_label = "neutral"

    return predicted_label

In [ ]:
def sentiment_classification_metric(example, pred, trace=None):
    predicted_label = detect_predicted_label(pred)

    # 检查预测结果是否与实际结果相同
    return example.sentiment.lower() == predicted_label

Create the DSPy Program

In [ ]:
import dspy
import openai
from dspy import OpenAI
import json

# openai.api_base = "https://api.gpts.vin/v1"
# openai.api_key = "sk-Pee7ZqyWixn655GY99Ff3eB6De9a4269Bf68E89505B90d48"

openai.api_base = "https://api.xiaoai.plus/v1"
openai.api_key = "sk-PZ3pu8tJkRrYv09o2aF29002270147809784D43e8241447d"


lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class AspectBasedSentimentAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    context = dspy.InputField()
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")

In [ ]:
import time
txt_file_url = "/content/drive/MyDrive/DsPy/Results/rest16/2ero-shot_BYD/2ero-shot_sentiment_analysis_results0-200.txt"

class AspectBasedSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(AspectBasedSentimentAnalysis)
        self.example_counter = 1
    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_context(sentence, aspect)

        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect, context=context)
        print("prediction:",prediction)
        return dspy.Prediction(context=context, sentiment=prediction.sentiment)
    def get_constrain_context(self, sentence, aspect):

        prompt = (
            f"# Your task is to answer the following questions with 'Yes' or 'No' and input the answer into the slot $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is positive? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is negative? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is neutral? $PRED$\n"
        )

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            n=1,
            stop=None
        )
        time.sleep(2)  # 添加1秒的时间间隔
        constrain_context = response.choices[0].message['content'].strip()
        return constrain_context
    def get_intro_context(self, aspect):
        time.sleep(2)  # 添加1秒的时间间隔
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please breifly introduce '{aspect}' in 250 words or less.."}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        intro_context = response.choices[0].message['content'].strip()
        return intro_context
    def get_analogies_context(self, aspect):
        prompt = (
            f"Recall three Aspect-Based Sentiment Analysis examples with the aspect '{aspect}' included in the sentence. Each example should include:\n1. The sentence to be analyzed.\n2. The predicted sentiment polarity of the aspect ('positive', 'neutral', 'negative')."
        )

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            n=1,
            stop=None
        )
        time.sleep(2)  # 添加1秒的时间间隔
        analogies_context = response.choices[0].message['content'].strip()
        return analogies_context
    def get_context(self, sentence, aspect):
        constrain_context = self.get_constrain_context(sentence, aspect)
        intro_context = self.get_intro_context(aspect)
        analogies_context = self.get_analogies_context(aspect)
        return f"# Constrain context:{constrain_context}\n\n# Background context:{intro_context}\n\n# Analogies context:{analogies_context}"
    def analyze_sentiment(self, sentence, aspect, context):
          prompt = (
                f"# Aspect: {aspect}\n"
                f"# Context: {context}\n"
                f"# Sentence: {sentence}\n"
                f"# Please analyze the sentiment polarity (positive, neutral, negative) of '{aspect}' in the sentence:\n"
            )
          response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=50,
                n=1,
                stop=None
            )
          time.sleep(2)  # 添加1秒的时间间隔
          sentiment = response.choices[0].message['content'].strip()
          return sentiment

    # 存储细节包括context、aspect、sentence、预测的sent
    def write_to_file(self, aspect, context, sentence, prediction):
        with open(txt_file_url, 'a') as f:
            example_number = f"example{self.example_counter:02d} :"
            f.write(f"{example_number}\n")
            f.write(f"Aspect: {aspect}\n")
            f.write(f"@Sentence: {sentence}\n")
            f.write(f"Context: {context}\n")
            f.write("----------------------------------------------------\n")
            f.write(f"@@Sentiment: {prediction.sentiment}\n")
            f.write("++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            self.example_counter += 1

Cauculate Acc of BYD-ABSA

In [ ]:
# 初始化情感分析器
analyzer = AspectBasedSentimentAnalyzer()

# 预测情感并计算准确率
correct_predictions = 0

for example in dev:
    try:
        prediction = analyzer(example['sentence'], example['aspect'])
        predicted_label = detect_predicted_label(prediction)
        actual_label = example.sentiment

        if predicted_label == actual_label:
            correct_predictions += 1
    except Exception as e:
        print(f"Error processing example: {example}")
        print(f"Exception: {str(e)}")
        continue
# for example in dev:
#     prediction = analyzer(example['sentence'], example['aspect'])
#     predicted_label = detect_predicted_label(prediction)
#     gold_label = example['sentiment']

#     if predicted_label == gold_label:
#         correct_predictions += 1

# 计算准确率
accuracy = correct_predictions / len(dev)

# 输出准确率并写入文件
dataset_name = "laptop14"  # 修改为实际的数据集名称
method_name = "byd"  # 修改为实际的方法名称

output_file_path = f'/content/drive/MyDrive/DsPy/Results/laptop14/{dataset_name}_{method_name}_accuracy1.txt'
with open(output_file_path, 'w') as f:
    f.write(f"Correct Predictions: {correct_predictions}\n")
    f.write(f"Total Predictions: {len(dev)}\n")
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")

prediction: Prediction(
    rationale='analyze the sentiment. The statement "OSX Mountain Lion soon to upgrade to Mavericks" does not provide any explicit sentiment towards OSX Mountain Lion. It simply states a future upgrade to Mavericks.',
    sentiment='neutral'
)
prediction: Prediction(
    rationale='produce the sentiment. We have the mention of the new MacBook Pro and the mention of "making the same mistake" with regard to USB ports. The use of "can\'t believe" also suggests a negative sentiment towards the lack of USB ports on the new model.',
    sentiment='negative'
)
prediction: Prediction(
    rationale='produce the sentiment. We can infer that the speaker is referring to a device or equipment that quickly becomes active when they open the lid. This quick response time could be seen as convenient and efficient, leading to a positive sentiment towards the aspect "wakes."',
    sentiment='Positive'
)
prediction: Prediction(
    rationale='produce the sentiment. We see that the

Zero-shot: Just run all testset.

In [ ]:
import time
from openai.error import APIError
# 创建一个AspectBasedSentimentAnalyzer的实例
analyzer = AspectBasedSentimentAnalyzer()

# 记录gold&pred的json文件和记录细则的txt文件
sentiment_comparison = {}

for i, example in enumerate(dev):
    result = analyzer(example['sentence'], example['aspect'])
    #analyzer.write_to_file(example['aspect'], result.context, example['sentence'], result) # 记录细则

    example_key = f"example{i+1:02d}"
    sentence = example['sentence']
    original_sentiment = example['sentiment']
    predicted_label = detect_predicted_label(result)
    sentiment_comparison[example_key] = {
        'sentence': sentence,
        'aspect': example['aspect'],
        'original_sentiment': original_sentiment,
        'predicted_sentiment': predicted_label
    }

    # Save the sentiment comparison dictionary to a .json file
output_path = '/content/drive/MyDrive/DsPy/Results/rest16/2ero-shot_BYD/sentiment_comparison.json'
with open(output_path, 'w') as f:
    json.dump(sentiment_comparison, f, indent=4)

print(f"Sentiment comparison saved to {output_path}")
# Print the sentiment comparison dictionary
print(sentiment_comparison)

prediction: Prediction(
    rationale='produce the sentiment. We can see that the sentence is expressing a positive sentiment towards the aspect of sushi. The words "really good" indicate a strong positive opinion about the quality of the sushi being served.',
    sentiment='positive'
)
prediction: Prediction(
    rationale='determine the sentiment. The word "adequate" suggests that the portions are satisfactory or sufficient, which implies a neutral sentiment. Since the sentence doesn\'t express extreme positivity or negativity, it seems to be neutral towards the aspect of portions.',
    sentiment='neutral'
)
prediction: Prediction(
    rationale='produce the sentiment. We can infer from the statement "is a must" that the speaker highly recommends the Green Tea creme brulee, indicating a positive sentiment.',
    sentiment='positive'
)
prediction: Prediction(
    rationale='produce the sentiment. We can infer the sentiment by considering the expression "can\'t say enough". This likel

Compile the Program

In [ ]:
# Compile the Program
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(AspectBasedSentimentAnalyzer(), trainset=train)

# optimized_classifier.save("/content/drive/MyDrive/DsPy/Results/laptop14/optimizer.json")

 20%|██        | 1/5 [00:11<00:46, 11.56s/it]

prediction: Prediction(
    rationale='produce the sentiment. We can see that the statement "sarah palin is awesome" indicates a positive sentiment towards Sarah Palin. Additionally, the response "yes she is" further confirms the positive sentiment. Therefore, based on the context and the agreement of the interlocutors, we can conclude that the sentiment towards Sarah Palin in this sentence is positive.',
    sentiment='positive'
)


 40%|████      | 2/5 [00:20<00:30, 10.05s/it]

prediction: Prediction(
    rationale='produce the sentiment. The sentence "russell brand and katy perry suit each other" implies that the relationship between Russell Brand and Katy Perry is well-matched. It does not provide any specific sentiment towards Katy Perry herself, so it\'s difficult to determine the sentiment polarity towards her.',
    sentiment='neutral'
)


 60%|██████    | 3/5 [00:30<00:19,  9.92s/it]

prediction: Prediction(
    rationale='produce the sentiment. We see that the sentence implies that Lindsay Lohan may feel bad about herself because Samantha Ronson has more followers on Twitter. This could potentially indicate a feeling of insecurity or low self-worth.',
    sentiment='negative'
)


 80%|████████  | 4/5 [00:40<00:09,  9.87s/it]

prediction: Prediction(
    rationale="produce the sentiment. In this sentence, the aspect 'Steve Jobs' is being compared to someone else, suggesting similarity or likeness in terms of having a vision. Therefore, the sentiment can be interpreted as positive, as it implies that 'Steve Jobs' is being seen as a positive figure in terms of having a vision.",
    sentiment='positive'
)


100%|██████████| 5/5 [00:49<00:00,  9.90s/it]

prediction: Prediction(
    rationale='produce the sentiment. We see that the sentence expresses a positive sentiment towards Google, as the speaker loves the company as a whole with all its services.',
    sentiment='positive'
)
Bootstrapped 2 full traces after 5 examples in round 0.


Evaluation

In [ ]:
from dspy.evaluate import Evaluate
import contextlib
import io
# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
# evaluate(optimized_classifier)
evaluation_output_file = f'/content/drive/MyDrive/DsPy/Results/rest16/rest16_BYD_evaluation.txt'
# 评估并记录结果
try:
    with open(evaluation_output_file, 'w') as f, contextlib.redirect_stdout(f):
        evaluate(optimized_classifier)
except Exception as e:
    print("An error occurred during evaluation:", e)

Average Metric: 176 / 219  (80.4): 100%|██████████| 219/219 [09:02<00:00,  2.48s/it]
